In [25]:

def get_ladder_json(word_length):
    assert word_length >= 3 and word_length <=6

    import requests
    import json
    r = requests.get(f'https://raw.githubusercontent.com/shaunhegarty/word-ladder/master/wordladder/resources/ladder-common-{word_length}.json')
    data = json.loads(r.content.decode('utf-8'))
    return data

data = get_ladder_json(3)

In [ ]:
list(data.keys())[:20]

In [ ]:
x = sorted(list(data.keys()), key=lambda x: len(data[x][0]), reverse=True)
x[:20]

In [ ]:
[','.join(l) for l in data.get('award-style')]

In [ ]:
for pair, ladder in [(pair, ladder) for pair, ladder in data.items()][:1]:
    print(pair, ladder[0])
    

In [ ]:
import sys
sys.path.append('/workspace/')

from sqlalchemy.orm import Session

from anagrammer import models
from anagrammer.database import engine

models.Base.metadata.create_all(bind=engine)

In [ ]:
with Session(engine) as session:
    results = session.query(models.Dictionary).filter(models.Dictionary.frequency > 50000).all()
    print(results)

In [16]:
from sqlalchemy import func

with Session(engine) as session:
    results = session.query(models.Dictionary, 
                            func.rank().over(order_by=models.Dictionary.frequency.desc())
                            ).filter(models.Dictionary.dictionary == 'common').all()
    score = {word.word: rank for word, rank in results}

In [ ]:
from sqlalchemy.dialects.postgresql import insert

with Session(engine) as session:
    # Populate Word Score
    values = []
    for word, word_score in score.items():
        values.append({'word': word, 'dictionary': 'common', 'score': word_score})
    statement = (
        insert(models.WordScore).values(values).on_conflict_do_nothing()
    )
    session.execute(statement)
    session.commit()

In [9]:
def sort_word_pair(pair):
    return '-'.join(sorted(pair.split('-')))
unique_ladder_keys = {sort_word_pair(pair) for pair in data.keys()}

In [18]:
def ladder_difficulty(ladder):
    word_score = 0
    for word in ladder:
        word_score += score.get(word)
    return word_score

In [26]:
from sqlalchemy.dialects.postgresql import insert

for word_length in range(3, 7):
    data = get_ladder_json(word_length=word_length)
    unique_ladder_keys = {sort_word_pair(pair) for pair in data.keys()}

    print(f'{len(unique_ladder_keys)} ladder keys')
    values = []
    for key in unique_ladder_keys:
        ladder_list = data.get(key)
        for index, ladder in enumerate(ladder_list):
            values.append({'pair': key,
                'dictionary': 'common',
                'chain': ','.join(ladder),
                'difficulty': ladder_difficulty(ladder),
                'variations': len(ladder_list),
                'variant': index, })
    print(f'Updating Database with {len(values)} records')
    with Session(engine) as session:
        statement = (
            insert(models.Ladder).values(values).on_conflict_do_nothing()
        )
        session.execute(statement)
        session.commit()

3637 ladder keys
Updating Database with 18078 records
3015 ladder keys
Updating Database with 23748 records
2166 ladder keys
Updating Database with 12056 records
481 ladder keys
Updating Database with 5665 records
